# 作成した埋め込みベクトル .npy ファイルからグラフを作成する

- ブレゼンハムのアルゴリズムを改良
- 線の太さを調整可能
- 画像サイズ、パディングを調整可能
- グラデーションを適用

## 1. 必要ライブラしをインポート

In [13]:
from PIL import Image
from tqdm import tqdm
import numpy as np
import os

size = 224
padding = 3
linewidth = 2

## 1.1 画像生成部分（だいたい以前のものを流用）

In [28]:
def map_range(x, a, b, c, d):
    """
    変数 $x$ を 範囲 [$a$, $b$] から [$c$, $d$] にリマップします
    :param x: 変換する対象
    :param a: 変換前の下端
    :param b: 変換前の上端
    :param c: 変換後の下端
    :param d: 変換後の上端
    :return: リマップした値
    """
    return (x - a) / (b - a) * (d - c) + c


def draw_point(canvas, x: int, y: int, width: int, intensity: int):
    """
    点を描画します
    :param canvas: numpy の2次元配列
    :param x: 座標 $x$
    :param y: 座標 $y$
    :param width: 点の幅
    :param intensity: 点の白色の強度
    :return:
    """

    size = canvas.shape[0]

    for i in range(-width, width + 1):
        yi = y + i

        if 0 <= yi < size:
            for j in range(-width, width + 1):
                xi = x + j

                if 0 <= xi < size:
                    r = np.sqrt(i ** 2 + j ** 2)  # 中心からの距離

                    if r <= width:  # 半径内部かどうかを判定
                        val = int(intensity * (1 - r / width))  # 直線的に色を変化
                        # val = int((intensity / (r_0 + 1)) - 1)  # 中心から反比例するように色を変化

                        if val > canvas[yi, xi]:  # 画素よりも大きい値であれば上書き
                            canvas[yi, xi] = val


def drawline_with_bresenham_algorithm(canvas, x0: int, y0: int, x1: int, y1: int, width: int, intensity: int):
    """
    ブレゼンハムのアルゴリズムで2点間の直線を描画します
    :param canvas: numpy の2次元配列
    :param x0: 点 $x_0$
    :param y0: 点 $y_0$
    :param x1: 点 $x_1$
    :param y1: 点 $y_1$
    :param width 直線の幅
    :return: None
    """

    # ここで float を int に変換
    x0 = round(x0)
    y0 = round(y0)
    x1 = round(x1)
    y1 = round(y1)

    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        draw_point(canvas, x0, y0, width, intensity)

        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * err

        if e2 > -dy:
            err -= dy
            x0 += sx

        if e2 < dx:
            err += dx
            y0 += sy

    draw_point(canvas, x1, y1, width, intensity)


def generate_graph(vectors, filename, size, padding, width):
    """
    埋め込みベクトルからグラフ表示画像を作成します
    :param vectors: アミノ酸配列
    :param filename: 保存するファイル名
    :param size: 画像サイズ
    :param padding: 画像のパディング
    :param width: 線幅
    :return:
    """

    x = y = 0
    x_min = x_max = 0
    y_min = y_max = 0

    points = [{"x": 0, "y": 0}]

    for vec in vectors:
        x += vec[0]
        y += vec[1]

        points.append({"x": x, "y": y})

        x_min, x_max = min(x, x_min), max(x, x_max)
        y_min, y_max = min(y, y_min), max(y, y_max)

    mapped_points = [
        {
            "x": map_range(point["x"], x_min, x_max, padding, (size - padding)),
            "y": map_range(point["y"], y_min, y_max, padding, (size - padding))
        }
        for point in points
    ]

    canvas = np.zeros((size, size), dtype=np.uint8)
    n_segments = len(mapped_points) - 1

    for i in range(n_segments):
        start, end = mapped_points[i], mapped_points[i + 1]

        progress = i / n_segments
        intensity = int(255 * progress)
        # draw.line([(start["x"], start["y"]), (end["x"], end["y"])], fill=255, width=3)
        drawline_with_bresenham_algorithm(canvas, start["x"], start["y"], end["x"], end["y"], width, intensity)

    img = Image.fromarray(canvas, mode="L")  # numpy 配列から画像を生成
    img.save(filename)

## 2. データ読み込み

In [29]:
model_name = "esm1b"  # モデル名
method_name = "umap2"  # 次元削減の手法

vectors_dir = f"../data/embedding-vectors/{model_name}/{method_name}_scaled_10_170"  # 埋め込みベクトルの場所
save_path = f"../graphs/{model_name}/{method_name}"  # グラフ表示画像の保存先

files = [os.path.join(vectors_dir, f) for f in os.listdir(vectors_dir) if f.endswith(".npy")]  # .npy ファイルを全て取得
files.sort()  # 一応ソート

os.makedirs(save_path, exist_ok=True)

In [30]:
for filename in tqdm(files):
    embedding_vectors = np.load(filename, mmap_mode="r")

    base = os.path.basename(filename).replace(f"_{method_name}_scaled_10_170.npy", ".png")
    save_filename = os.path.join(save_path, base)

    generate_graph(embedding_vectors, save_filename, size, padding, linewidth)  # グラフ表示画像を作成

  0%|                                                                                          | 0/7716 [00:00<?, ?it/s]/tmp/ipykernel_84726/784737333.py:132: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(canvas, mode="L")  # numpy 配列から画像を生成
100%|███████████████████████████████████████████████████████████████████████████████| 7716/7716 [13:04<00:00,  9.84it/s]
